**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
# Install tensorflow, keras, opencv, skvideo and ffmpeg
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act indicates what action $a_t$ should be taken at a specific state $s_t$ according to the policy (behaviour) that the agent is following. In fact, this action will be taken with probability 1-$\epsilon$ while a random action among the four possible ones will be taken with probability $\epsilon$. So the lower $\epsilon$, the less we explore other options and the more we select the best action spotted so far. 

The above occurs in the case where $train=True$. Otherwise, we simply follow the best action empowered by the policy and do not explore other possibilities, which is why $\epsilon$ is here for. Indeed, RL is known for the existing trade-off between exploitation and exploration. We need to exploit the agent’s current best option while continuing to explore new possibilities, not suggested by the current policy. This especially allows the agent to avoid getting stuck with a non-optimal behaviour. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train= 51 # set small when debugging
epochs_test= 11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The ```position```array is of dimension $(N+2)\times(N+2)$. Note that we have increased the board size by 4 in both width and height, so 2 on each side. This implies that the position array reads 0 everywhere on the original board size except on the rat's position where it reads 1. It contains -1 on the outside of the orginal board, meaning on the elements added to the inital array (2 cells wide border), which surround the original board. This gives the rat, which sees at a distance of 2, an incentive to stay within the delimited bounds of the board. 

Regarding the ```board``` array, it has identical dimensions (to the position array) and represents the expanded board. It is filled with either -1, 0 or 0.5 if the cell contains respectively poison, nothing and cheese. It is generated with random variables following a binomial law and serves to give rewards to the rat regarding the cell he moves to. Note that once the board has helped update the reward, the cell is set to 0.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, 1)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        # Save as a mp4
        if e%10 == 0: 
            env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [169]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.0/13.0. Average score (-4.0)
Win/lose count 12.0/12.0. Average score (-2.0)
Win/lose count 6.5/9.0. Average score (-2.1666666666666665)
Win/lose count 9.0/10.0. Average score (-1.875)
Win/lose count 9.0/10.0. Average score (-1.7)
Win/lose count 12.5/11.0. Average score (-1.1666666666666667)
Win/lose count 10.5/27.0. Average score (-3.357142857142857)
Win/lose count 14.0/17.0. Average score (-3.3125)
Win/lose count 4.0/4.0. Average score (-2.9444444444444446)
Win/lose count 13.0/20.0. Average score (-3.35)
Win/lose count 9.0/8.0. Average score (-2.9545454545454546)
Win/lose count 9.5/10.0. Average score (-2.75)
Final score: -2.75


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




1) 

$Q^\pi(s,a) = E_{\pi}[\sum_0^T \gamma^t r(s_t,a_t) | s_0 = s, a_0 = a]$ 

$Q^\pi(s,a) = E_{\pi} [r(s,a) + \sum_{1\leq t \leq T} \gamma^t r(s_t,a_t) | s_0=s, a_0=a]$

$Q^\pi(s,a) = r(s,a) + \sum_{s'} P_{ss'}^a \gamma E_{\pi} [\sum_0^T \gamma^t r(s_{t+1}, a_{t+1}) | s_1 = s']$

$Q^\pi(s,a) = r(s,a) + \sum_{s'} P_{ss'}^a \sum_{a'} \pi(s',a') \gamma E_{\pi} [\sum_0^T \gamma^t r(s_{t+1}, a_{t+1}) | s_1 = s', a_1 = a'] $

$Q^\pi(s,a) = r(s,a) + \sum_{s'} P_{ss'}^a \sum_{a'} \pi(s',a') \gamma Q^\pi(s',a') $

$Q^\pi(s,a) = E_{(s',a')\sim p(.|s,a)} [r(s,a) + \gamma Q^\pi(s',a')]$


2) 

$Q^*(s,a) = max_\pi Q^\pi(s,a) = max_\pi [r(s,a) + \sum_{s'} P_{ss'}^a \sum_{a'} \pi(s',a') \gamma Q^\pi(s',a')]$

$Q^*(s,a) = max_\pi Q^\pi(s,a) = r(s,a) + \sum_{s'} P_{ss'}^a max_\pi [\sum_{a'} \pi(s',a') \gamma Q^\pi(s',a')]$

$Q^*(s,a) = r(s,a) + \sum_{s'} P_{ss'}^a \gamma max_{a'} Q^*(s',a')$

$Q^*(s,a) = E_{s' \sim \pi(.|s,a)} [r(s,a) + \gamma max_{a'} Q^*(s',a')]$ 


3) 

$L(\theta) = E_{s'\sim \pi^*(.|s,a)} || r + \gamma max max_{a'} Q(s',a',\theta) - Q(s,a,\theta) ||^2  $

which can be rewritten as 

$L(\theta) = E_{s'\sim \pi^*(.|s,a)} || (r - \sum_{s'} P_{ss'}^a R_{ss'}^a) + \gamma (max max_{a'} Q^*(s',a',\theta) - E_{(s',a') \sim p(.|s,a)}[Q^\pi(s',a',\theta)] )  ||^2  $

From this equation, it appears that the right hand term in the norm will be a small positive number, probably negligible in the long run. The left hand side component, on the opposite, is the difference between the actual and expected rewards we can obtain at the next time step. This quantity can have a much bigger impact on the loss function so the individual, by maximising the loss, will be encouraged to maximise its immediate reward (LHS), without compromising too much its future state value function (supposing he will take the best action) (RHS). 

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [8]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        # stores move
        self.memory.insert(0, m)
        if len(self.memory) > self.max_memory: 
            self.memory.pop(self.max_memory)

    def random_access(self):
        n = np.random.randint(len(self.memory))
        return self.memory[n]

***
The pipeline we will use for training is given below:

In [9]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [10]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # Uniquely needs to defined what action you take given current state  
        # Select the action adviced by neural network
        # Reshape s (add 1 in front because expecting (batch_size,5,5,2))
        return np.argmax(self.model.predict(s.reshape(1, *s.shape))[0])
    

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
   
    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            (envstate, envstate_next, action, reward, game_over_) = self.memory.random_access()
            input_states[i, :, :, :] = envstate
            if game_over_:
                target_q[i, action] = reward
            else:
                Q_sa = np.max(self.model.predict(envstate_next.reshape(1, *envstate_next.shape))[0])
                target_q[i, action] = reward + self.discount * Q_sa
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)
        
        return l
    
        
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten())
        model.add(Dense(64))
        model.add(Dense(128))
        model.add(Dense(64))
        model.add(Dense(self.n_action))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
epochs_train = 30
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.01, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/030 | Loss 0.0070 | Win/lose count 1.5/4.0 (-2.5)
Epoch 001/030 | Loss 0.0000 | Win/lose count 0/0 (0)
Epoch 002/030 | Loss 0.0080 | Win/lose count 0.5/0 (0.5)
Epoch 003/030 | Loss 0.0041 | Win/lose count 2.0/2.0 (0.0)
Epoch 004/030 | Loss 0.0003 | Win/lose count 0.5/0 (0.5)
Epoch 005/030 | Loss 0.0002 | Win/lose count 0.5/0 (0.5)
Epoch 006/030 | Loss 0.0002 | Win/lose count 1.5/1.0 (0.5)
Epoch 007/030 | Loss 0.0025 | Win/lose count 1.5/0 (1.5)
Epoch 008/030 | Loss 0.0007 | Win/lose count 3.0/0 (3.0)
Epoch 009/030 | Loss 0.0004 | Win/lose count 1.5/0 (1.5)
Epoch 010/030 | Loss 0.0004 | Win/lose count 10.0/2.0 (8.0)
Epoch 011/030 | Loss 0.0008 | Win/lose count 4.0/1.0 (3.0)
Epoch 012/030 | Loss 0.0099 | Win/lose count 5.0/3.0 (2.0)
Epoch 013/030 | Loss 0.0038 | Win/lose count 2.5/1.0 (1.5)
Epoch 014/030 | Loss 0.0019 | Win/lose count 4.0/2.0 (2.0)
Epoch 015/030 | Loss 0.0059 | Win/lose count 6.5/0 (6.5)
Epoch 016/030 | Loss 0.0089 | Win/lose count 7.5/0 (7.5)
Epoch 017/030 | L

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(4, (3, 3), padding="same", activation="relu"))
        model.add(Conv2D(8, (3, 3), padding="same", activation="relu"))
        model.add(Conv2D(16, (3, 3), padding="same", activation="relu"))
        model.add(Flatten())
        model.add(Dense(32))
        model.add(Dense(self.n_action, activation ='linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/030 | Loss 0.0219 | Win/lose count 13.0/18.0 (-5.0)
Epoch 001/030 | Loss 0.0124 | Win/lose count 10.0/2.0 (8.0)
Epoch 002/030 | Loss 0.0169 | Win/lose count 3.5/4.0 (-0.5)
Epoch 003/030 | Loss 0.0117 | Win/lose count 3.5/3.0 (0.5)
Epoch 004/030 | Loss 0.0099 | Win/lose count 6.0/5.0 (1.0)
Epoch 005/030 | Loss 0.0262 | Win/lose count 1.5/2.0 (-0.5)
Epoch 006/030 | Loss 0.0107 | Win/lose count 3.0/4.0 (-1.0)
Epoch 007/030 | Loss 0.0027 | Win/lose count 2.0/3.0 (-1.0)
Epoch 008/030 | Loss 0.0172 | Win/lose count 5.5/1.0 (4.5)
Epoch 009/030 | Loss 0.0065 | Win/lose count 6.0/9.0 (-3.0)
Epoch 010/030 | Loss 0.0044 | Win/lose count 6.0/4.0 (2.0)
Epoch 011/030 | Loss 0.0039 | Win/lose count 2.0/0 (2.0)
Epoch 012/030 | Loss 0.0067 | Win/lose count 3.0/2.0 (1.0)
Epoch 013/030 | Loss 0.0075 | Win/lose count 5.0/4.0 (1.0)
Epoch 014/030 | Loss 0.0065 | Win/lose count 4.0/2.0 (2.0)
Epoch 015/030 | Loss 0.0153 | Win/lose count 5.5/3.0 (2.5)
Epoch 016/030 | Loss 0.0123 | Win/lose count 10.0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [16]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 5.5/0. Average score (5.5)
Win/lose count 5.5/3.0. Average score (4.0)
Win/lose count 5.0/3.0. Average score (3.3333333333333335)
Win/lose count 2.0/1.0. Average score (2.75)
Win/lose count 5.5/3.0. Average score (2.7)
Win/lose count 2.0/1.0. Average score (2.4166666666666665)
Win/lose count 4.0/2.0. Average score (2.357142857142857)
Win/lose count 5.0/1.0. Average score (2.5625)
Win/lose count 1.0/1.0. Average score (2.2777777777777777)
Win/lose count 6.0/4.0. Average score (2.25)
Win/lose count 3.5/3.0. Average score (2.090909090909091)
Final score: 2.090909090909091
Test of the FC
Win/lose count 4.0/3.0. Average score (1.0)
Win/lose count 11.5/8.0. Average score (2.25)
Win/lose count 3.0/1.0. Average score (2.1666666666666665)
Win/lose count 16.5/10.0. Average score (3.25)
Win/lose count 10.0/4.0. Average score (3.8)
Win/lose count 3.5/2.0. Average score (3.4166666666666665)
Win/lose count 8.5/5.0. Average score (3.4285714285714284)
Win/lose count 3.5/

In [17]:
HTML(display_videos('cnn_test10.mp4'))

In [18]:
HTML(display_videos('fc_test10.mp4'))

The cascade of fully connected layer performs better than the CNN in this case. It not only eats more cheese but also avoids the poison better. Nevertheless, their performance are quite similar in general, and are strongly correlated to the structure of the map created. 

In both cases (especially when you take more epochs), you can see a clear improvement of the score, which proves that the bot learns correctly. Note that if its score varies a lot, it mainly depends on the initial position of the bot. If it is close to much cheese, the score is likely to be more positive. On this note, the temperature plays an important role as it changes the structure of the map. A higher temperature involves more cheese and therefore a higher score in average, although the behaviour of the bot is not strictly more desirable. 

Finally, we notice that the bot easily gets stuck and does not explore the full map. We will see in the following part how to correct this default. 

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [53]:
def train_explore(agent,env,epoch,prefix=''):
    
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            
            discount_epsilon = 0.9 # rate of decrease for exploration
            agent_epsilon = agent.epsilon * discount_epsilon # modify agent epsilon
            agent.set_epsilon(agent_epsilon) # pass modified epsilon to agent  
            
            action = agent.act(state) # get action 

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    

In [59]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        self.malus_position = np.zeros((self.grid_size, self.grid_size)) # define malus 

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale = 16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        # self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        # Depreciate malus and discount
        self.malus_discount = 0.9
        self.malus_position *= self.malus_discount # depreciate malus
        self.malus_position[self.malus_position < 0.35] = 0 # no malus if state was not recently visited
        
        # Incorporate malus to recently visited states 
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.8 # set malus of state just visited to -0.5 (for next time)
        
        reward = reward + self.board[self.x, self.y]
        
        self.board[self.x, self.y] = 0 # reset this state value to 0 as you ate what could be on this cell 
        game_over = self.t > self.max_time
        
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                    self.board.reshape(self.grid_size, self.grid_size,1),
                            self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus
        
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros([self.grid_size, self.grid_size])

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                    self.board.reshape(self.grid_size, self.grid_size,1),
                            self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [35]:
epochs_train = 51
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.8, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore50.mp4'))

Epoch 000/051 | Loss 0.0609 | Win/lose count 9.5/69.0395712000001 (-59.5395712000001)
Epoch 001/051 | Loss 0.0654 | Win/lose count 6.5/63.1425576000001 (-56.6425576000001)
Epoch 002/051 | Loss 0.0348 | Win/lose count 6.5/37.60886960000001 (-31.108869600000013)
Epoch 003/051 | Loss 0.0412 | Win/lose count 14.0/44.31150960000003 (-30.31150960000003)
Epoch 004/051 | Loss 0.0300 | Win/lose count 7.5/23.93862959999999 (-16.43862959999999)
Epoch 005/051 | Loss 0.0351 | Win/lose count 6.0/20.847999999999995 (-14.847999999999995)
Epoch 006/051 | Loss 0.0309 | Win/lose count 9.0/35.0951096 (-26.0951096)
Epoch 007/051 | Loss 0.0321 | Win/lose count 21.0/26.536869599999992 (-5.536869599999992)
Epoch 008/051 | Loss 0.0351 | Win/lose count 18.0/28.132465599999993 (-10.132465599999993)
Epoch 009/051 | Loss 0.0350 | Win/lose count 18.5/29.61341119999999 (-11.113411199999991)
Epoch 010/051 | Loss 0.0225 | Win/lose count 11.5/18.7113168 (-7.211316799999999)
Epoch 011/051 | Loss 0.0268 | Win/lose count 

In [36]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 17.5/35.6867712. Average score (-18.186771200000003)
Win/lose count 18.0/33.87150959999998. Average score (-17.029140399999992)
Win/lose count 18.0/45.722160000000045. Average score (-20.593480266666678)
Win/lose count 26.0/39.71409840000001. Average score (-18.873634800000012)
Win/lose count 28.5/26.90643679999999. Average score (-14.780195200000009)
Win/lose count 22.5/25.65531679999999. Average score (-12.842715466666673)
Win/lose count 18.0/21.353284. Average score (-11.487082400000006)
Win/lose count 26.0/26.065759999999994. Average score (-10.059417100000005)
Win/lose count 24.5/28.47569839999999. Average score (-9.38344835555556)
Win/lose count 17.0/47.83585840000006. Average score (-11.52868936000001)
Win/lose count 22.0/30.54237119999999. Average score (-11.257205890909098)
Final score: -11.257205890909098


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***